In [1]:
# !uv pip install git+https://github.com/willccbb/verifiers.git

In [ ]:
import os
from speedy_utils import *
from llm_utils import *

In [3]:
client = MOpenAI(base_url='http://localhost:7999/v1')

In [4]:
lm = AsyncLM(port=7999)

Using model: Qwen/Qwen3-235B-A22B-Instruct-2507-FP8


In [5]:
vector_service = VectorCache(url_or_model='http://localhost:8000/v1')#, model_name='poly-embed-0.6b-250904')

Auto-detected backend: openai
Infer model name: poly-embed-0.6b-250904


In [7]:
# # from exps.embeding_dedup import filter_duplicated
# from glob import glob
# from tqdm import tqdm

# paths = glob('/home/anhvth5/projects/TRANSLATE_UI/assets/*/*/examples.csv')
# for path in tqdm(paths):
#     print(path)
#     df = pd.read_csv(path)
#     embeds = vector_service(df.source.tolist())
#     dedup_ids = filter_duplicated(embeds, 0.08)
#     list_gs = list(sorted(dedup_ids, key=len, reverse=True))
#     # map group id to df
#     for i, g in enumerate(list_gs):
#         df.loc[g, 'group_id'] = i
#     df.to_csv(path.replace('examples.csv', 'examples_dedup.csv'), index=False)        
#     print(path, len(df), len(list_gs))
#     break

In [8]:
df = df.drop_duplicates('source')

In [10]:
# grouped = df.groupby("group_id")
# dfs = [group for _, group in sorted(grouped, key=lambda x: len(x[1]), reverse=True)]

In [124]:
print('\n---------------\n'.join(dfs[200].source.tolist()))

타락의 상흔 1회 제거하기
---------------
타락의 상흔 6회 제거하기
---------------
타락의 상흔 12회 제거하기


In [ ]:
g = list_gs[21000]
df.iloc[g]

,source,target
241799,你已经在围场如鱼得水了，打开有概率获得净瓶、念珠、吴山石、沧海玉、灵荒璧、宝匣钥匙、表情包等...,"Ngươi đã ở bãi săn như cá gặp nước, mở có xác ..."
253535,在围场成长的见证，打开有概率获得净瓶、念珠、吴山石、沧海玉、灵荒璧、宝匣钥匙、表情包等珍贵道...,"Chứng kiến trưởng thành ở bãi săn, mở có xác s..."


In [11]:
paths = glob('/home/anhvth5/projects/TRANSLATE_UI/assets/*/*/examples_*.csv')
paths

['/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS9_CFL_KO_TH/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS9_CFL_ZH_TH/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS9_MS_ZH_EN/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS9_CFL_KO_ZH/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS2_TTLCY_ZH_VI/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS9_RWS_ZH_VI/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS3_TD_ZH_EN/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS3_TN_ZH_VI/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS1_KTO_ZH_VI/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/TSN_FW2_ZH_EN/examples_dedup.csv',
 '/home/anhvth5/projects/TRANSLATE_UI/assets/LC_STANDARD/GS9_CFL_KO_VI/examples_d

In [12]:
df = pd.read_csv(paths[0])
df['group_id'] = df['group_id'].astype(int)
df = df.set_index('group_id')
df.loc[200]

,source,target
group_id,,
200,"모든 적에게 공격력의 <NegativeSkill>158%</>만큼 피해를 주고, 7...",สร้าง DMG แก่ศัตรูทั้งหมด<NegativeSkill>158%</...
200,"모든 적에게 공격력의 <NegativeSkill>173%</>만큼 피해를 주고, 7...",สร้าง DMG แก่ศัตรูทั้งหมด<NegativeSkill>173%</...
200,"모든 적에게 공격력의 <NegativeSkill>189%</>만큼 피해를 주고, 7...",สร้าง DMG แก่ศัตรูทั้งหมด<NegativeSkill>189%</...
